# SpineNet Lumbar Grading Example

Last edited: 05/07/2022

This notebook shows an example of using SpineNet to grade a typical T2 lumbar scan
on CUDA-enabled hardware. Note that it is also possible to run SpineNet on a CPU
only (however, this will be slower).


## 01. Loading In SpineNet + Data

In [33]:
!pip install keras-core
!pip install wurlitzer
!pip install tensorflow==2.15.0  # Downgrade TensorFlow if necessary
!pip install tensorflow-decision-forests
!pip check  # Optional: Check for any remaining conflicts

aiobotocore 2.13.0 has requirement aiohttp<4.0.0,>=3.9.2, but you have aiohttp 3.9.1.
apache-beam 2.46.0 has requirement dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8.
apache-beam 2.46.0 has requirement numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4.
apache-beam 2.46.0 has requirement pyarrow<10.0.0,>=3.0.0, but you have pyarrow 16.1.0.
beatrix-jupyterlab 2023.128.151533 has requirement jupyterlab~=3.6.0, but you have jupyterlab 4.2.1.
boto3 1.26.100 has requirement botocore<1.30.0,>=1.29.100, but you have botocore 1.34.106.
cloud-tpu-client 0.10 has requirement google-api-python-client==1.8.0, but you have google-api-python-client 2.131.0.
conda 24.5.0 has requirement packaging>=23.0, but you have packaging 21.3.
google-cloud-aiplatform 0.6.0a1 has requirement google-api-core[grpc]<2.0.0dev,>=1.22.2, but you have google-api-core 2.11.1.
google-cloud-automl 1.0.1 has requirement google-api-core[grpc]<2.0.0dev,>=1.14.0, but you have google-api-core 2.11.1.
jupyterlab 4.2.1 has require

In [2]:
!git clone --depth 1 https://github.com/rwindsor1/SpineNet

Cloning into 'SpineNet'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 54 (delta 0), reused 29 (delta 0), pack-reused 0
Unpacking objects: 100% (54/54), 1.85 MiB | 7.50 MiB/s, done.


In [3]:
!pip install "tensorflow<2.11"  #Sadece deneme, çalıstı
!python -c "import tensorflow as tf; print(tf.reduce_sum(tf.random.normal([1000, 1000])))"

^C
ERROR: Operation cancelled by user


In [5]:
import os

In [6]:
os.getcwd()

'/kaggle/working'

In [8]:
!git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 4245, done.
remote: Counting objects: 100% (4245/4245), done.
remote: Compressing objects: 100% (3236/3236), done.
remote: Total 4245 (delta 1193), reused 2120 (delta 938), pack-reused 0
Receiving objects: 100% (4245/4245), 48.86 MiB | 36.07 MiB/s, done.
Resolving deltas: 100% (1193/1193), done.


In [14]:
!cd models/research && protoc object_detection/protos/*.proto --python_out=.

In [15]:
os.environ['PYTHONPATH'] += ":/kaggle/working/models:/kaggle/working/models/research:/kaggle/working/models/research/slim"

In [16]:
!ls models/research/object_detection/protos

__init__.py			       keypoint_box_coder.proto
anchor_generator.proto		       keypoint_box_coder_pb2.py
anchor_generator_pb2.py		       losses.proto
argmax_matcher.proto		       losses_pb2.py
argmax_matcher_pb2.py		       matcher.proto
bipartite_matcher.proto		       matcher_pb2.py
bipartite_matcher_pb2.py	       mean_stddev_box_coder.proto
box_coder.proto			       mean_stddev_box_coder_pb2.py
box_coder_pb2.py		       model.proto
box_predictor.proto		       model_pb2.py
box_predictor_pb2.py		       multiscale_anchor_generator.proto
calibration.proto		       multiscale_anchor_generator_pb2.py
calibration_pb2.py		       optimizer.proto
center_net.proto		       optimizer_pb2.py
center_net_pb2.py		       pipeline.proto
eval.proto			       pipeline_pb2.py
eval_pb2.py			       post_processing.proto
faster_rcnn.proto		       post_processing_pb2.py
faster_rcnn_box_coder.proto	       preprocessor.proto
faster_rcnn_box_coder_pb2.py	       preprocessor_pb2.py
faster_rcnn_pb2.py		       region_simi

In [ ]:
%cd ..

In [27]:
!dir

README.md	    cvt_text	   lstm_object_detection  seq_flow_lite
adversarial_text    deep_speech    marco		  slim
attention_ocr	    deeplab	   nst_blogpost		  vid2depth
audioset	    delf	   object_detection
autoaugment	    efficient-hrl  pcl_rl
cognitive_planning  lfads	   rebar


In [23]:
os.getcwd()

'/kaggle/working/models'

In [26]:
%cd ./research

/kaggle/working/models/research


In [31]:
!pip install tf-slim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 kB 4.7 MB/s eta 0:00:0000:010:01


In [34]:
!pip install tf-slim keras keras-core tensorflow
!pip uninstall tensorflow -y
!pip install tensorflow

Found existing installation: tensorflow 2.15.0
Uninstalling tensorflow-2.15.0:
  Successfully uninstalled tensorflow-2.15.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 57.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 74.1 MB/s eta 0:00:00:00:01
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 23.5.26
    Uninstalling flatbuffers-23.5.26:
      Successfully uninstalled flatbuffers-23.5.26
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.1
    Uninstalling tensorboard-2.15.1:
      Successfully uninstalled tensorboard-2.15.1
  Attempting uninstall: k

In [38]:
!pip uninstall keras -y
!pip uninstall keras-core -y
!pip install tensorflow
!pip install --upgrade tf-models-official

Found existing installation: keras 3.4.1
Uninstalling keras-3.4.1:
  Successfully uninstalled keras-3.4.1
  Using cached keras-3.4.1-py3-none-any.whl.metadata (5.8 kB)
Using cached keras-3.4.1-py3-none-any.whl (1.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires tensorflow~=2.15.0, but you have tensorflow 2.17.0 which is incompatible.
tensorflow-text 2.15.0 requires tensorflow<2.16,>=2.15.0; platform_machine != "arm64" or platform_system != "Darwin", but you have tensorflow 2.17.0 which is incompatible.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.17.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━

In [43]:
from tensorflow.keras.layers import *

In [44]:
from object_detection.builders import model_builder

AttributeError: module 'tensorflow.keras.layers' has no attribute 'experimental'

In [40]:
!pip install --upgrade tensorflow

In [25]:
import sys
import os
from pathlib import Path
sys.path.insert(0, str(Path(os.getcwd()).parent)) # Add parent directory to path. This shouldn't be needed if SpineNet is pip-installed



import spinenet
from spinenet import SpineNet, download_example_scan
from spinenet.io import load_dicoms_from_folder

# download example scans 
os.makedirs('example_scans', exist_ok=True)

# download_example_scan('t2_lumbar_scan_1', file_path='example_scans')
# or use the other example scan available
scan_name = 't2_lumbar_scan_2'

example_scan_folder = f'./example_scans'
download_example_scan(scan_name, file_path=example_scan_folder)


# download weights from server. This may take a minute or two.
# You do not need to run this line if weights have already been downloaded.
spinenet.download_weights(verbose=True, force=False)

# load in spinenet. Replace device with 'cpu' if you are not using a CUDA-enabled machine.
spnt = SpineNet(device='cuda:0', verbose=True)




# metadata to be overwritten in the scan - useful if certain important values are missing from some/all dicom files
# in this case, slice thickness and image orientation are missing from the dicom files and so we add false values
# of 2mm and a sagittal orientation code ([0,1,0,0,0,1]). 
# Do not overwrite this metadata if it already exists in the dicom files being used.
overwrite_dict = {'SliceThickness': [2], 'ImageOrientationPatient': [0, 1, 0, 0, 0, -1]}


# loads in a dicom from the example scan folder.
# if set, the `require_extensions` flag requires that files end with `.dcm`
scan = load_dicoms_from_folder(f"{example_scan_folder}/{scan_name}", require_extensions=False, metadata_overwrites=overwrite_dict)


ModuleNotFoundError: No module named 'spinenet'

## 02. Visualize Scan Slices

Show each slice from the sagittal T2 lumbar scan and print out metadata information.

In [ ]:
print(f'Scan has {scan.volume.shape[-1]} sagittal slices, of dimension {scan.volume.shape[0]}x{scan.volume.shape[1]} ({scan.pixel_spacing} mm pixel spacing) and {scan.slice_thickness} mm slice thickness.')


import matplotlib.pyplot as plt
fig = plt.figure(figsize=(8,8))

# show each sagittal slice
for slice_idx in range(scan.volume.shape[-1]):
    ax = fig.add_subplot(4,3,slice_idx+1)
    ax.imshow(scan.volume[:,:,slice_idx], cmap='gray')
    ax.set_title(f'Slice {slice_idx+1}')
    ax.axis('off')


## 03. Detect Vertebrae
Use SpineNet to detect vertebrae and then show the detections in slices.

In [ ]:
# detect and identify vertebrae in scan. Note that pixel spacing information is required 
# so SpineNet knows what size to split patches into.
vert_dicts = spnt.detect_vb(scan.volume, scan.pixel_spacing)

print(f'{len(vert_dicts)} vertebrae detected; {[vert_dict["predicted_label"] for vert_dict in vert_dicts]}')


# visualize vertebrae detections in slices 
from matplotlib.patches import Polygon 
import numpy as np
fig = plt.figure(figsize=(8,8))
for slice_idx in range(scan.volume.shape[-1]):
    ax = fig.add_subplot(4,3,slice_idx+1)
    ax.imshow(scan.volume[:,:,slice_idx], cmap='gray')
    ax.set_title(f'Slice {slice_idx+1}')
    ax.axis('off')
    for vert_dict in vert_dicts:
        if slice_idx in vert_dict['slice_nos']:
            poly_idx = int(vert_dict['slice_nos'].index(slice_idx))
            poly = np.array(vert_dict['polys'][poly_idx])
            ax.add_patch(Polygon(poly, ec='y',fc='none'))
            ax.text(np.mean(poly[:,0]), np.mean(poly[:,1]), vert_dict['predicted_label'],c='y', ha='center',va='center')

fig.suptitle('Detected Vertebrae (all slices)')
plt.show()

## 04. Show vertebrae detections in the mid sagittal slice

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
# plot mid sagittal slice and detected vertebrae
plt.figure(figsize=(5,5))
plt.imshow(scan.volume[:,:,scan.volume.shape[-1]//2], cmap='gray')
for idx, vert_dict in enumerate(vert_dicts):
    if scan.volume.shape[-1]//2 in vert_dict['slice_nos']:
        poly = np.array(vert_dict['polys'][vert_dict['slice_nos'].index(scan.volume.shape[-1]//2)])
        plt.gca().add_patch(Polygon(poly, fc='none', ec='y'))
        plt.text(np.mean(poly[:,0]), np.mean(poly[:,1]), vert_dict['predicted_label'], color='y',fontsize=20, va='center', ha='center')
    else:
        continue

plt.axis('off')
plt.title('Detected Vertebrae (Mid Sagittal Slice)')
plt.show()


## 05. Perform grading of T2 scans for common radiological conditions

Since this is a T2 sagittal lumbar scan, SpineNet can be used to perform radiological grading for several common spinal degenerative conditions. Note that this is trained on IVDs from T12/L5 to L5/S1 so may not be accurate for vertebrae outside this range.

In [ ]:
# extract IVDs using the detections from the previous stage.
ivd_dicts = spnt.get_ivds_from_vert_dicts(vert_dicts, scan.volume)

# grade IVDs - note that this is only validated on IVDs from L5/S1 to T12/L5 vertebrae.
# IVDs gradings are output as a pandas dictionary. For information on the grading schemes used, see http://zeus.robots.ox.ac.uk/spinenet2/
ivd_grades = spnt.grade_ivds(ivd_dicts)
ivd_grades.head(len(ivd_dicts))



In [ ]:
results_folder = './results'
os.makedirs(results_folder, exist_ok=True)

ivd_grades.to_csv(f"{results_folder}/{scan_name}_ivd_grades.csv")


In [6]:
!pip install keras-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 9.1 MB/s eta 0:00:00ta 0:00:01


  Using cached keras-2.15.0-py3-none-any.whl.metadata (2.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.0 MB/s eta 0:00:00:00:0100:01
Using cached keras-2.15.0-py3-none-any.whl (1.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 2.3 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 16.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.0
    Uninstalling ml-dtypes-0.4.0:
      Successfully uninstalled ml-dtypes-0.4.0
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.0
    Uninstalling tensorboard-2.17.0:
      Successfully uninstalled tensorboard-2.17.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.0
    Uninstalling tensorflow-2.17.0:
      Successfu